In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "distilbert-base-uncased"  # Change this to your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [5]:
# Prepare dummy input
dummy_input = tokenizer("Hello, world!", return_tensors="pt").input_ids

# Export the model
torch.onnx.export(
    model,
    dummy_input,
    f"{model_name}.onnx",
    input_names=['input_ids'],
    output_names=['last_hidden_state'],
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence'},
        'last_hidden_state': {0: 'batch_size', 1: 'sequence'}
    },
    opset_version=11
)

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/transformers/models/distilbert/modeling_distilbert.py:215: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [6]:
import onnxruntime as ort
import numpy as np
from transformers import AutoTokenizer

# Load the ONNX model
ort_session = ort.InferenceSession("distilbert-base-uncased.onnx")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [9]:
# Prepare input
text = "Hello, world!"
encoded_input = tokenizer(text, return_tensors="np")

# Run inference
ort_inputs = {ort_session.get_inputs()[0].name: encoded_input.input_ids}
ort_outputs = ort_session.run(None, ort_inputs)

print("ONNX Model Output:", ort_outputs[0].shape)

ONNX Model Output: (1, 6, 768)
